# Data management

In [1]:
import pandas as pd

class OptionDataParser:
    @staticmethod
    def parse_option_row(row, ref_date=None):
        """
        Extrait la date d'échéance, le strike et la volatilité implicite d'une ligne de DataFrame.
        La fonction se base sur la colonne 'Ticker' pour extraire la date et le strike.

        :param row: une ligne du DataFrame
        :param ref_date: date de référence pour calculer le temps à expiration (par défaut, aujourd'hui)
        :return: (time_to_expiry, strike, vol)
        """
        # Utilise la date d'aujourd'hui comme référence si non précisée
        if ref_date is None:
            ref_date = pd.Timestamp.today()

        # Extraction depuis la colonne 'Ticker'
        # Exemple de format attendu : "AAPL 3/21/25 C207.5"
        ticker_str = row['Ticker']
        try:
            tokens = ticker_str.split()
            # La deuxième valeur correspond à la date d'échéance
            expiry_str = tokens[1]
            # La troisième valeur contient le type et le strike (ex: "C207.5")
            option_info = tokens[2]
            # On extrait le strike en enlevant la première lettre
            strike = float(option_info[1:])

            # Conversion de la date d'échéance en datetime (format supposé mm/dd/yy)
            expiry_date = pd.to_datetime(expiry_str, format='%m/%d/%y')
            # Calcul du temps jusqu'à expiration en années
            time_to_expiry = (expiry_date - ref_date).days / 365.0

            # Récupération de la volatilité implicite : on essaye 'VIM' puis 'VIM.1'
            vol = row.get('VIM')
            if pd.isna(vol):
                vol = row.get('VIM.1')
            # On suppose que vol est déjà en format numérique (sinon conversion nécessaire)
            return time_to_expiry, strike, vol
        except Exception as e:
            print(f"Erreur lors du parsing de la ligne : {ticker_str} - {e}")
            return None, None, None

    @staticmethod
    def prepare_option_data(file_path):
        """
        Lit le fichier Excel et prépare un DataFrame avec les colonnes nécessaires :
        'maturity' (temps en années), 'strike' et 'vol'.
        On ignore la première ligne de données si elle contient des informations d'en-tête.

        :param file_path: chemin du fichier Excel
        :return: DataFrame avec colonnes ['maturity', 'strike', 'vol']
        """
        # Lecture en indiquant que la deuxième ligne contient les noms de colonnes
        data = pd.read_excel(file_path, header=1)

        # Optionnel : filtrer sur une option (par exemple, uniquement sur AAPL)
        # data = data[data['Ticker'].str.contains("AAPL", na=False)]

        # Initialisation d'une liste pour stocker les données extraites
        rows = []
        # Parcours des lignes
        for idx, row in data.iterrows():
            # On ignore les lignes sans valeur dans 'Ticker' ou qui ne sont pas des chaînes
            if pd.isna(row['Ticker']) or not isinstance(row['Ticker'], str):
                continue
            t_exp, strike, vol = OptionDataParser.parse_option_row(row)
            if t_exp is not None and strike is not None and vol is not None:
                rows.append({'maturity': t_exp, 'strike': strike, 'vol': vol})

        df_options = pd.DataFrame(rows)
        # Optionnel : filtrer pour garder uniquement les maturités positives
        df_options = df_options[df_options['maturity'] > 0]
        return df_options


In [6]:
# Description: This file contains the data classes for SVI parameters and calibration parameters.
from dataclasses import dataclass
from typing import List

# ---------------- SVIParams Class ----------------
@dataclass
class SVIParams:
    strike: float
    maturity: float
    spot: float

@dataclass
class MarketDataPoint:
    strike: float
    maturity: float
    implied_volatility: float

@dataclass
class SVICalibrationParams:
    opt_data: List[MarketDataPoint]
    spot: float


In [21]:
pd.read_excel("../data_options/options_data_TSLA 2.xlsx", header=1)

,Ticker,Strike,Bid,Ask,Dern,VIM,Vol.,Ticker.1,Strike.1,Bid.1,Ask.1,Dern.1,VIM.1,Vol..1
0,21-Mar-25 (7j); TailleC 100; R 4.56; FwdI 239.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TSLA 3/21/25 C235,235.0,14.900000,15.100000,14.900000,85.680920,8057.0,TSLA 3/21/25 P235,235.0,8.900000,9.050000,9.000000,85.071276,20176.0
2,TSLA 3/21/25 C237.5,237.5,13.450000,13.650000,13.600000,84.914692,5577.0,TSLA 3/21/25 P237.5,237.5,10.000000,10.150000,10.020000,84.755103,11729.0
3,TSLA 3/21/25 C240,240.0,12.150000,12.300000,12.220000,84.460158,19609.0,TSLA 3/21/25 P240,240.0,11.150000,11.300000,11.110000,84.139071,22352.0
4,TSLA 3/21/25 C242.5,242.5,10.900000,11.050000,10.970000,83.943920,10922.0,TSLA 3/21/25 P242.5,242.5,12.400000,12.550000,12.410000,83.639923,4466.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,TSLA 1/15/27 C230,230.0,91.449997,92.199997,91.500000,62.401750,15.0,TSLA 1/15/27 P230,230.0,64.250000,64.849998,66.129997,62.431761,36.0
86,TSLA 1/15/27 C235,235.0,89.500000,90.349998,90.419998,62.306934,39.0,TSLA 1/15/27 P235,235.0,67.099998,67.750000,69.000000,62.327806,21.0
87,TSLA 1/15/27 C240,240.0,87.750000,88.550003,86.940002,62.284228,72.0,TSLA 1/15/27 P240,240.0,70.050003,70.699997,67.820000,62.249438,0.0
88,TSLA 1/15/27 C245,245.0,86.199997,86.800003,86.400002,62.331813,4.0,TSLA 1/15/27 P245,245.0,72.849998,73.699997,76.190002,62.091568,17.0


# Test

In [8]:
file_path = "../data_options/options_data_TSLA 2.xlsx"
df_options = OptionDataParser.prepare_option_data(file_path)

# Convertir le DataFrame en liste de MarketDataPoint
opt_data = [
    MarketDataPoint(strike=row['strike'], maturity=row['maturity'], implied_volatility=row['vol'])
    for _, row in df_options.iterrows()
]

Erreur lors du parsing de la ligne : 21-Mar-25 (7j); TailleC 100; R 4.56; FwdI 239.13 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 17-Apr-25 (34j); TailleC 100; R 4.56; FwdI 240.04 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 16-May-25 (63j); TailleC 100; R 4.56; FwdI 240.91 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 20-Jun-25 (98j); TailleC 100; R 4.56; FwdI 242.01 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 18-Jul-25 (126j); TailleC 100; R 4.53; FwdI 242.93 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 15-Aug-25 (154j); TailleC 100; R 4.50; FwdI 243.89 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 19-Sep-25 (189j); TailleC 100; R 4.46; FwdI 244.96 - could not convert string to float: 'ailleC'
Erreur lors du parsing de la ligne : 21-Nov-25 (252j); TailleC 100;

In [22]:
df_options

,maturity,strike,vol
5,0.032877,230.0,74.473853
6,0.032877,235.0,73.782707
7,0.032877,240.0,73.259895
8,0.032877,245.0,72.646909
9,0.032877,250.0,72.201319
...,...,...,...
70,1.780822,230.0,62.401750
71,1.780822,235.0,62.306934
72,1.780822,240.0,62.284228
73,1.780822,245.0,62.331813


In [17]:
import numpy as np

opt_data

import plotly.graph_objects as go

# Extract unique values for maturity and strike
maturities = sorted(list(set(point.maturity for point in opt_data)))
strikes = sorted(list(set(point.strike for point in opt_data)))

# Create meshgrid for surface plot
X, Y = np.meshgrid(maturities, strikes)
Z = np.zeros_like(X)

# Fill Z matrix with volatility values
for i, strike in enumerate(strikes):
    for j, maturity in enumerate(maturities):
        for point in opt_data:
            if point.strike == strike and point.maturity == maturity:
                Z[i,j] = point.implied_volatility
                break

# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(
    title='Implied Volatility Surface',
    scene = dict(
        xaxis_title='Maturity',
        yaxis_title='Strike',
        zaxis_title='Implied Volatility'
    )
)
fig.show(renderer="browser")


In [20]:
Z

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , 64.06786929, 63.5689218 ,  0.        ,
        62.968596  , 62.24157127,  0.        ,  0.        ],
       [74.47385285, 73.40639539, 69.3822204 , 67.79499502, 67.59447168,
        64.82106158, 64.35720959, 63.67940002, 63.21266323, 62.80647249,
        62.72110312, 62.49204821, 62.46080129, 62.40174954],
       [73.78270713, 72.90383739, 68.97404412, 67.43018245, 67.29695883,
        64.56471285, 64.15274619,  0.        ,  0.        , 62.67383695,
         0.        ,  0.        , 62.46853243, 62.30693383],
       [73.25989462, 72.38751029, 68.54379449, 67.15860651, 67.01745065,
        64.33536947, 63.98673982, 63.32420001, 62.78369626, 62.55917713,
        62.51336246, 62.30920107, 62.39332062, 62.28422847],
       [72.64690918, 71.98472673, 68.24727917, 66.8433861 , 66.79845347,
        64.13401195, 63.89154459,  0.        ,  0.        , 62.46324812,
         0.        ,  0.  